<a href="https://colab.research.google.com/github/SahanaRaghu/Machine-Learning-cookbook/blob/master/myWork_MultiClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from google.colab import files
uploaded = files.upload()

Saving Input_dpci_pog_preprocessed.csv to Input_dpci_pog_preprocessed (2).csv


In [21]:
from google.colab import files
uploadedTest = files.upload()

Saving Input_dpci_pog_test_data.csv to Input_dpci_pog_test_data (1).csv


In [0]:
#Import to dataframe
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pandas as pd
import io
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

input_data = pd.read_csv(io.StringIO(uploaded['Input_dpci_pog_preprocessed.csv'].decode('utf-8')))
test_data = pd.read_csv(io.StringIO(uploadedTest['Input_dpci_pog_test_data.csv'].decode('utf-8')))

input_data = input_data.reindex(
    np.random.permutation(input_data.index))

In [0]:
def preprocess_features(input_data_dataframe, length):
  """Prepares input features from data set.

  Args:
    input_data_dataframe: A Pandas DataFrame expected to contain data
      from the data set.
  Returns:
    A DataFrame that contains the features to be used for the model, including
    synthetic features.
  """

  features = pd.DataFrame()
  features["lengthBy10"] = input_data_dataframe["data"].apply(lambda l: len(l)/10)
  
  #length = input_data_dataframe['data'].apply(lambda l: len(l)).max()
  i = 1
  while(i<=length):
    features[str(i)] = input_data_dataframe['data'].apply(lambda l: 1 if (str.isdigit(l[i-1:i]) and l[i-1:i] != "") else 0 if (l[i-1:i] == "") else 2)
    i = i+1
  
  return features

def func(x):
    if x=='dpci':
        return 0
    elif x=='pog':
        return 1
    elif x == 'backroomlocation':
        return 2
    elif x == 'DCLabels':
        return 3
    else:
        return 4

def preprocess_targets(input_data_dataframe):
  """Prepares target features (i.e., labels) from data set.

  Args:
    input_data_dataframe: A Pandas DataFrame expected to contain data
      from the data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = pd.DataFrame()
  output_targets['data_type'] = input_data_dataframe['target']
  output_targets['data_type'] = output_targets['data_type'].apply(func)

  return output_targets

In [24]:
#---Divide data into training & validation data---

length = input_data['data'].apply(lambda l: len(l)).max()
# Choose the first 28 out of 40 examples for training.
training_examples = preprocess_features(input_data.head(28), length)
training_targets = preprocess_targets(input_data.head(28))

# Choose the last 12 out of 40 examples for validation.
validation_examples = preprocess_features(input_data.tail(12), length)
validation_targets = preprocess_targets(input_data.tail(12))

#Test data
test_examples = preprocess_features(test_data, length)
test_targets = preprocess_targets(test_data)

# Double-check that we've done the right thing.
print("Training examples summary:")
display.display(training_examples.head())
print("Validation examples summary:")
display.display(validation_examples.head())

print("Training targets summary:")
display.display(training_targets.head())

print("Validation targets summary:")
display.display(validation_targets.head())

print("Test data summary:")
display.display(test_examples.head())
display.display(test_targets.head())

Training examples summary:


,lengthBy10,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
19,0.7,2,1,1,1,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0
32,0.3,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,0.7,2,1,1,1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0
13,0.7,2,1,1,1,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0
20,1.0,2,1,1,2,1,1,1,2,1,1,0,0,0,0,0,0,0,0,0


Validation examples summary:


,lengthBy10,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
8,0.9,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
0,0.7,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
6,0.9,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
14,0.7,2,1,1,1,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0
16,0.7,2,1,1,1,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0


Training targets summary:


...

Validation targets summary:


,data_type
8,0
0,0
6,0
14,1
16,1


Test data summary:


,lengthBy10,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.3,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.7,2,1,1,1,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0.9,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
3,0.4,2,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,2,1,1,2,1,1,1,2,1,1,0,0,0,0,0,0,0,0,0


,data_type
0,4
1,1
2,0
3,4
4,2


In [0]:
def construct_feature_columns(input_features):
  """Construct the TensorFlow Feature Columns.

  Args:
    input_features: The names of the numerical input features to use.
  Returns:
    A set of feature columns
  """ 
  return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])

In [0]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a neural network model.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
    
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [0]:
import os
import time

def get_new_path(name=""):
    base = os.path.abspath("/Users/z073862/Documents/ML/myWork/model")
    logpath = os.path.join(base, name + "_" + str(int(time.time())))
    print("Logging to {}".format(logpath))
    return logpath

In [0]:
# Training model using linear classification model of Tensor flow

def train_linear_classifier_model(
    learning_rate,
    steps,
    batch_size,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
  """Trains a linear classification model.
  
  In addition to training, this function also prints training progress information,
  as well as a plot of the training and validation loss over time.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    training_examples: A `DataFrame` containing one or more columns from
      `california_housing_dataframe` to use as input features for training.
    training_targets: A `DataFrame` containing exactly one column from
      `california_housing_dataframe` to use as target for training.
    validation_examples: A `DataFrame` containing one or more columns from
      `california_housing_dataframe` to use as input features for validation.
    validation_targets: A `DataFrame` containing exactly one column from
      `california_housing_dataframe` to use as target for validation.
      
  Returns:
    A `LinearClassifier` object trained on the training data.
  """

  periods = 10
  steps_per_period = steps / periods
  
  # Create a linear classifier object.
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0) 
  linear_classifier = tf.estimator.LinearClassifier(
    feature_columns=construct_feature_columns(training_examples),
    optimizer=my_optimizer,
    n_classes=5,
    model_dir=get_new_path("tfmodel")
  )
  
  # Create input functions.
  training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["data_type"], 
                                          batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                  training_targets["data_type"], 
                                                  num_epochs=1, 
                                                  shuffle=False)
  predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                    validation_targets["data_type"], 
                                                    num_epochs=1, 
                                                    shuffle=False)
  
  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("LogLoss (on training data):")
  training_log_losses = []
  validation_log_losses = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    linear_classifier.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
    
    # Evaluate accuracy on Validation data set.
    accuracy_score_val = linear_classifier.evaluate(input_fn=predict_validation_input_fn, 
                                         steps=steps)["accuracy"]
    print('\nAccuracy: {0:f}'.format(accuracy_score_val))
   

  return linear_classifier

In [29]:
linear_classifier = train_linear_classifier_model(
    learning_rate=0.3,
    steps=50,
    batch_size=5,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

Logging to /Users/z073862/Documents/ML/myWork/model/tfmodel_1547037847
Training model...
LogLoss (on training data):

Accuracy: 0.916667

Accuracy: 0.916667

Accuracy: 0.916667

Accuracy: 0.916667

Accuracy: 0.916667

Accuracy: 0.916667

Accuracy: 1.000000

Accuracy: 0.916667

Accuracy: 0.916667

Accuracy: 1.000000


In [0]:
  def evalute_test_data(
    steps,
    test_examples,
    test_targets):
    
    predict_test_input_fn = lambda: my_input_fn(test_examples, 
                                                      test_targets["data_type"], 
                                                      num_epochs=1, 
                                                      shuffle=False)

    # Evaluate accuracy on Test data set.
    accuracy_score_test = linear_classifier.evaluate(input_fn=predict_test_input_fn, 
                                         steps=steps)["accuracy"]
    print('\nAccuracy: {0:f}'.format(accuracy_score_test))
    
    predictions = linear_classifier.predict(input_fn=predict_test_input_fn)

    for prediction in predictions:
        print("Predictions:    {} with probabilities {}\n".format(
            prediction["classes"], prediction["probabilities"]))  
        
    return accuracy_score_test 


In [31]:
evalute_test_data(
    50,
    test_examples,
    test_targets)


Accuracy: 0.833333
Predictions:    [b'4'] with probabilities [0.00487401 0.00671028 0.0032084  0.00321418 0.9819932 ]

Predictions:    [b'1'] with probabilities [4.4687547e-02 9.4988078e-01 8.8703376e-04 7.7738788e-04 3.7672520e-03]

Predictions:    [b'0'] with probabilities [0.9166971  0.01806615 0.0520684  0.0071182  0.00605012]

Predictions:    [b'4'] with probabilities [0.00838519 0.00327146 0.0053222  0.00284744 0.9801737 ]

Predictions:    [b'2'] with probabilities [2.0398738e-02 5.0966628e-04 9.7683388e-01 9.1425376e-04 1.3434726e-03]

Predictions:    [b'3'] with probabilities [5.2130339e-04 3.1486699e-05 1.3136999e-04 9.9910825e-01 2.0763077e-04]



0.8333333

In [32]:
# Approach 1 - no error but does not save the model using input_receiver_fn

# linear_classifier.export_savedmodel(
#     export_dir = '/Users/z073862/Documents/ML/myWork/model/',
#     serving_input_receiver_fn=
#     assets_extra=None,
#     as_text=False,
#     checkpoint_path=None,
#     strip_default_attrs=False)


export_dir = '/Users/z073862/Documents/ML/myWork/model/'

feature_spec = tf.feature_column.make_parse_example_spec(construct_feature_columns(training_examples))

input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

linear_classifier.export_savedmodel(export_dir, input_receiver_fn, as_text=False) 

b'/Users/z073862/Documents/ML/myWork/model/1547037891'

In [33]:
# Approach 2 - no error but does not save the model using serving_input_receiver_fn

export_dir = '/Users/z073862/Documents/ML/myWork/model/'


feature_spec = tf.feature_column.make_parse_example_spec(construct_feature_columns(training_examples))
serving_input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
export_dir = linear_classifier.export_savedmodel(export_dir, serving_input_receiver_fn=serving_input_receiver_fn)
print('Exported to {}'.format(export_dir))

Exported to b'/Users/z073862/Documents/ML/myWork/model/1547037892'


In [0]:
def serving_input_receiver_fn():
  """An input receiver that expects a serialized tf.Example."""
  serialized_tf_example = tf.placeholder(dtype=tf.string,
                                         shape=[20],
                                         name='input_example_tensor')
  receiver_tensors = {'examples': serialized_tf_example}
  features = tf.parse_example(serialized_tf_example, feature_spec)
  return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)



In [43]:
# Save the model
linear_classifier.export_savedmodel(
    '/Users/z073862/Documents/ML/myWork/model',
    serving_input_receiver_fn=serving_input_receiver_fn)

b'/Users/z073862/Documents/ML/myWork/model/1547038354'

In [34]:
training_examples.head

<bound method NDFrame.head of     lengthBy10  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  \
19         0.7  2  1  1  1  2  2  2  0  0   0   0   0   0   0   0   0   0   0   
32         0.3  2  1  1  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   
12         0.7  2  1  1  1  2  1  1  0  0   0   0   0   0   0   0   0   0   0   
13         0.7  2  1  1  1  2  1  2  0  0   0   0   0   0   0   0   0   0   0   
20         1.0  2  1  1  2  1  1  1  2  1   1   0   0   0   0   0   0   0   0   
..         ... .. .. .. .. .. .. .. .. ..  ..  ..  ..  ..  ..  ..  ..  ..  ..   
4          0.8  1  1  1  1  1  1  1  1  0   0   0   0   0   0   0   0   0   0   
10         0.7  2  1  1  1  2  1  2  0  0   0   0   0   0   0   0   0   0   0   
31         0.3  2  1  1  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   
46         1.9  2  2  1  1  1  1  1  1  1   1   1   1   1   1   1   2   2   2   
22         1.0  2  1  1  2  1  1  1  2  1   1   0   0   0   0   0   0   0   0  

In [35]:
serving_input_receiver_fn

<function tensorflow.python.estimator.export.export.build_parsing_serving_input_receiver_fn.<locals>.serving_input_receiver_fn>